In [105]:
import os, sys
import glob
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser

#Covarage matrix as csv file without qoutes, seperated by commas
amplPos = "/home/pato/Skrivebord/HPV_subtyping/FASTQ/amplPos1.bed"

# Housegenes to check and remove
houseGeneEndRow = 5

bedPoolN = 2

# Location of all valid reference names that can be given
ref = "/home/pato/Skrivebord/HPV_subtyping/References"

# Find possible references to match
faF = [f for f in listdir(path) if isfile(join(path, f))]
faFiles = []
for i in faF:
    if "fasta.fai" not in i:
        faFiles.append(i)
        
# Location of where to save output files
outpath = "/home/pato/Skrivebord/HPV_subtyping/"
logFile = "/home/pato/Skrivebord/HPV_subtyping/Results/testlog.txt"

In [106]:
ampl_df = pd.read_csv(amplPos,sep="\t", header = None)
ampl_df = ampl_df.iloc[int(houseGeneEndRow):,]


In [107]:
ampl_df[['Pool', 'Submitted_reg']] = ampl_df[5].str.split(';', 1, expand=True)

ampl_df

,0,1,2,3,4,5,Pool,Submitted_reg
5,HPV6LR_X00203,167,247,AMPL4425925,.,Pool=1;SUBMITTED_REGION=HPV6_E6_1,Pool=1,SUBMITTED_REGION=HPV6_E6_1
6,HPV6LR_X00203,361,448,AMPL4426005,.,Pool=2;SUBMITTED_REGION=HPV6_E6_2,Pool=2,SUBMITTED_REGION=HPV6_E6_2
7,HPV6LR_X00203,585,673,AMPL4425967,.,Pool=2;SUBMITTED_REGION=HPV6_E7_1,Pool=2,SUBMITTED_REGION=HPV6_E7_1
8,HPV6LR_X00203,776,866,AMPL4426080,.,Pool=1;SUBMITTED_REGION=HPV6_E7_2,Pool=1,SUBMITTED_REGION=HPV6_E7_2
9,HPV6LR_X00203,2884,2969,AMPL4425974,.,Pool=2;SUBMITTED_REGION=HPV6_E2_1,Pool=2,SUBMITTED_REGION=HPV6_E2_1
...,...,...,...,...,...,...,...,...
216,HPV97pHR_DQ080080,722,810,AMPL4425973,.,Pool=1;SUBMITTED_REGION=HPV97_E7_2,Pool=1,SUBMITTED_REGION=HPV97_E7_2
217,HPV97pHR_DQ080080,2861,2952,AMPL4426024,.,Pool=1;SUBMITTED_REGION=HPV97_E2_1,Pool=1,SUBMITTED_REGION=HPV97_E2_1
218,HPV97pHR_DQ080080,3268,3343,AMPL4426068,.,Pool=1;SUBMITTED_REGION=HPV97_E2_2,Pool=1,SUBMITTED_REGION=HPV97_E2_2
219,HPV97pHR_DQ080080,3383,3482,AMPL4425894,.,Pool=1;SUBMITTED_REGION=HPV97_E2_3,Pool=1,SUBMITTED_REGION=HPV97_E2_3


In [109]:
bedpool = []
for pool in ampl_df.Pool.unique():
    poolN = pool.split("=")[1]
    savename = "pool"+str(poolN)+".bed"
    #Filtering for pool
    filtered = ampl_df[ampl_df['Pool']==pool]
    # Cutting down to only hpvtype, start and end positions
    filtered = filtered.iloc[:,0:3]

    # Now create bed file with corrected hpvtype names for each pool
    corrected_hpvtypes = []
    for index, row in filtered.iterrows():
        hpvtype = row[0]
        contig_srt = row[1]
        contig_end = row[2]
        # Create dataframe with correct ID's
        # Append _ to hpv name so that HPV161 does not match HPV16 in a search:
        if "_" in hpvtype:
            ht = hpvtype.split("_")[1]
            pass
        else:
            ht = hpvtype + "_" 

        # search list of fasta files for id
        ref = [i for i in faFiles if ht in i]
        # Checking that there is only one match, then saving without .fasta in name
        if len(ref) > 1:
            corrected_hpvtypes.append(["MultipleRefsFound", contig_srt, contig_end])
        else:
            # Checking that there is atleast one match
            if ref == []:
                ref = "No_type_found"
            else:
                ref = ref[0].replace(".fasta","")
                corrected_hpvtypes.append([ref, contig_srt, contig_end])

    df = pd.DataFrame(corrected_hpvtypes)
    df.to_csv(outpath+"/"+savename, sep="\t", header = None, index = None)